# Twitter: Complex

## Summary 

### Data 
* Degree distributions 
* Activity distributions
    * Tweet
    * Favourite 
    * Follow

### Types 

#### Tweet 

* `id` $\in V \subset \mathbb{N}$            
* `user_id` $\in \mathbb{N}$         
* `retweet_status` $\in \mathbb{B}$  
* `retweeted_id` $\in \mathbb{N}$ 
* `retweeted_user_id` $\in \mathbb{N}$ 
* `content` $\in \mathbb{R}$ 
* `favorite_count` $\in \mathbb{N}$
* `retweet_count` $\in \mathbb{N}$  
* `step` $\in \mathbb{N}$        

#### User 

* `id` $\in \mathbb{N}$       
* `pos` $\in \mathbb{N}$              
* `feed` $\subset \mathcal{T}$  
* `opinion` $\in \mathbb{R}$       
* `tweet_rate` $\in \mathbb{R}$     
* `retweet_rate` $\in \mathbb{R}$     
* `favorite_rate` $\in \mathbb{R}$   
* `follow_rate` $\in \mathbb{R}$     
* `unfollow_rate` $\in \mathbb{R}$    
* `tweets` $\subset \mathcal{T}$  
* `favorites` $\subset \mathcal{T}$  

### Model 

#### Parameters

* Population size $N$
* Number of replicates $R$
* Number of time steps $T$
* Controversialness $\alpha$ 
* Follow threshold $f$
* Unfollow threshold $u$

#### Properties 

* Population size `N` 
* Time step `step` 
* Tweet ID counter `tweet_id` 
* Tweet objects `tweets` 
* Controversialness `α`
* Follow threshold `follow_threshold`
* Unfollow threshold `unfollow_threshold`

#### Initialization 

* `id` has been initialized sequentially 
* `pos` has been initialized sequentially 
* `feed` has been initialized as an empty array              
* `opinion` has been initialized as drawn from $\mathcal{U}(-1,1)$          
* `tweet_rate` and `retweet_rate` have been initialized with a sample of 2012 activity data 
* `favorite_rate` has been initialized with a sample of 2012 activity data 
* `follow_rate` and `unfollow_rate` have been initialized with a sample of 2012 activity data 
* `tweets` has been initialized as an empty array
* `favorites` has been initialized as an empty array
* `follower_graph` has been constructed such that the friends (outneighbors) of each user (node) have been drawn from a $\Gamma(2,1)$

#### Calibration 

Not discussed yet. 

#### Validation 

Not discussed yet.

#### Micro-Dynamics (`agent_step!`)

* The minimal opinion dynamics (`Update!`) follows the equation by [Baumann et al. (2020)](https://doi.org/10.1103/PhysRevLett.124.048301) 
* The minimal follow dynamics (`Follow!`) consists in a uniform sampling from the complement of set of friends according to the user's `follow_rate` and `follow_threshold` 
* The minimal unfollow dynamics (`Unollow!`) consists in a uniform sampling from the set of friends according to the user's `unfollow_rate` and `unfollow_threshold` 

#### Macro-Dynamics (`model_step!`)

* Time step `step` is incremented by one at each iteration
* Tweet ID counter `tweet_id` is incremented by one every time a tweet is authored (or retweeted)
* Tweet objects `tweets` is enriched with every tweet object authored (or retweeted)

## Thoughts & Doubts  
* How to define distinct time scales ($T_{F} \gg T_{RT} > T_{L}$)
* What about defining a status type as a reduced representation of the status object
* Choose `nsteps` taking into account the constraints induced by Monitor limits 
* highlight the latency of the opinion variable: users do not have direct access to the *cognitive opinion* of other but  they have indirect access to *behavioral opinion* leading to a non-zero (tipically high) probability of mis-interpretation (we may model it via a portfolio of probability distributions centered around the true value)
* think about how to implement possibile polarization/segregation mitigation strategies (e.g. *homophily-induced heterophily*)
* Can we learn anything significant from Higgs2012 static and temporal graphs
* How to reliably mine temporal behavioral opinion and reaction mechanisms (e.g. net positive interaction rate: net of T,RTs,L of known polarity / total number of interactions) 

## To-Do List
* Overwrite all graph-related functions in Agents to ensure compatibility with `MultiplexGraphs.jl` (e.g. `get_node_agents()`,...)
* Think about all elementary variables, parameters and timing distributions
* Explore the portfolio of possible recommendation systems (e.g. GNN, bipartite, centrality-based, simple weighted sum,...)
* Implement basic `Follow!` and `RT!` behaviors
* Implement the simplest recommendation system (in `model_step!`)
* Implement non-trivial interpretation distribution $\rho_i$
* Explore and evaluate Higgs2012 static and temporal graph data
* Think of the best parametrization that allows effective calibration 

## Modules

In [1]:
using Agents
#using LightGraphs, SimpleWeightedGraphs
using Random
using GraphPlot
using Distributions
using DrWatson
using BenchmarkTools
using StatsBase
using Plots
using DataFrames

# Costum Modules
MG = include("/Users/Pit/GitHub/Econophysics/Project/Models/ourModels/Pietro&Riccardo/MultiplexGraphs.jl")

Main.MultiplexGraphs

## Agent Type

In [2]:
mutable struct User <: AbstractAgent
    id::Int                # idenfier code ∈ ℕ
    pos::Int               # nodal position ∈ ℕ
    login::Bool            # activity binary state ∈ 𝔹
    tweets::Array          # tweet history 
    favorites::Array       # favorite history 
    feed::Array            # tweet feed
    effective_feed::Array  # effective feed of tweets to be read 
    opinion::Real          # leaning ∈ [-1,1] 
    attention::Int         # cardinality effective feed
    epsilon_like::Real     # favorite threshold
    epsilon_RT::Real       # RT threshold
    epsilon_follow::Real   # follow threshold
    login_rate::Real        
    logout_rate::Real
    tweet_rate::Real       # probability to tweet its opinion if active
end

## Tweet Type

In [3]:
mutable struct Tweet
    id::Int 
    user_id::Int
    retweet_status::Bool
    retweeted_id::Int
    retweeted_user_id::Int
    content::Real
    favorite_count::Int
    retweet_count::Int
    step::Int
end

## Utilities

In [4]:
function node_neighbors(agent::User, model::ABM{User,GraphSpace{Main.MultiplexGraphs.MultiplexGraph{Int64}},typeof(fastest),Dict{Symbol,Any}}, which::Symbol; neighbor_type::Symbol=:default) 
    @assert neighbor_type ∈ (:default, :in, :out)
    if neighbor_type == :default
        MG.neighbors(model.space.graph, agent.pos, which)
    elseif neighbor_type == :in
        MG.inneighbors(model.space.graph, agent.pos, which)
    elseif neighbor_type == :out
        MG.outneighbors(model.space.graph, agent.pos, which)
    end
end

# Power law sampling
function PowerlawSample(a::Real, b::Real, γ::Real)
    a + (b-a)*rand()^γ
end;
#round(Int, PowerlawSample(1,1000,2.2))

function get_tweet_from_id(id::Int, model)
    tweet=[t for t in model.tweets if t.id == id]
    return tweet[1]
end;

### Initialization 

In [5]:
function InitializeModel(N)
    step=0
    tweet_id=0
    tweets=[]
    properties = @dict(N,step,tweet_id,tweets) # or @dict(N) where @dict is a "macro" which is equivalent to  Dict(:N => N)
    space=GraphSpace(MG.MultiplexGraph(N))
    model = ABM(User, space; properties) #AgentType, space; properties (dict)
    
    # Populate the model
    for id in 1:N
        pos = id
        login=false
        tweets=[]
        favorites=[]
        feed=[]
        effective_feed=[]
        opinion = rand(Uniform(-1, 1))
        attention = round(Int, PowerlawSample(1,N,2.5))
        epsilon_like = rand(Uniform(0, 1)) # Remember to check for polarity alignment
        epsilon_RT = rand(Uniform(0, 0.1))
        epsilon_follow = rand(Uniform(0, 0.01))
        login_rate = rand(Uniform(0, 1))
        logout_rate = rand(Uniform(0, 1))
        tweet_rate=rand(Uniform(0, 1))
        add_agent!(pos, model, login, tweets, favorites, feed, effective_feed, opinion, attention, epsilon_like, epsilon_RT, epsilon_follow, login_rate, logout_rate, tweet_rate) # even though opinion is optional
    end
    
    agents=[a for a in allagents(model)]
    # Initialize follower graph (static)
    for agent in agents
        others=[a for a in agents if a !=agent]
        friends=StatsBase.sample(others, rand(1:length(others)); replace=false, ordered=false) # replace=true for favorite graph
        for friend in friends
            MG.add_edge!(model.space.graph, agent.pos, friend.pos, :follower_graph)
        end
    end
    
    # Initialize feeds
    friends=StatsBase.sample(agents, rand(1:round(Int, length(agents)/3)) ; replace=false, ordered=false) # replace=true for favorite graph
    tweet_id=1
    for friend in friends 
        append!(friend.tweets, model.tweet_id)
        followers_pos=node_neighbors(friend, model, :follower_graph, neighbor_type=:in) 
        followers=[f for f in agents if f.pos in followers_pos]
        tweet=Tweet(tweet_id, friend.id, false, 0, 0, friend.opinion, 0, 0, model.step)
        for follower in followers
            push!(follower.feed, tweet)
        end
        push!(model.tweets, tweet)
        model.tweet_id+=1
    end
    
    # Shuffle feeds 
    for agent in agents 
        agent.feed[randperm(length(agent.feed))]
    end
    
    return model
end

InitializeModel (generic function with 1 method)

### Macro-Dynamics

In [6]:
function model_step!(model)
    model.step+=1
end

model_step! (generic function with 1 method)

### Micro-Dynamics

In [7]:
# Login dynamics 
function Login!(agent,model)
    if agent.login==false && rand()≤agent.login_rate
        agent.login=true
    end
end

# Read dynamics 
function Read!(agent, model)
    if agent.attention ≤ length(agent.feed)
        agent.effective_feed=agent.feed[1:agent.attention]
    else
        agent.effective_feed=agent.feed
    end
end

# Tweet dynamics
function Tweet!(agent,model)
    if rand() ≤ agent.tweet_rate
        followers_pos=node_neighbors(agent, model, :follower_graph, neighbor_type=:in) 
        followers=[f for f in allagents(model) if f.pos in followers_pos]
        tweet=Tweet(model.tweet_id, agent.id, false, 0, 0, agent.opinion, 0, 0, model.step)
        for follower in followers
            push!(follower.feed, tweet)
        end 
        append!(agent.tweets, model.tweet_id)
        push!(model.tweets, tweet)
        model.tweet_id+=1
    end
end

# Like dynamics
function Like!(agent,model) 
    for tweet in agent.effective_feed 
        if tweet.id ∉ agent.favorites
            if abs(agent.opinion-tweet.content) ≤ agent.epsilon_like
                tweet.favorite_count+=1
                append!(agent.favorites, tweet.id)
                MG.add_edge!(model.space.graph, agent.pos, tweet.user_id, :favorite_graph)
            end
        end
    end
end

# Retweet dyamics 
function RT!(agent,model) 
    for tweet in agent.effective_feed
        if tweet.retweet_status==false 
            if abs(agent.opinion-tweet.content) ≤ agent.epsilon_RT
                tweet.retweet_count+=1
                MG.add_edge!(model.space.graph, agent.pos, tweet.user_id, :retweet_graph)
                
                followers_pos=node_neighbors(agent, model, :follower_graph, neighbor_type=:in) 
                followers=[f for f in allagents(model) if f.pos in followers_pos]
                retweet=Tweet(model.tweet_id, agent.id, true, tweet.id, tweet.user_id, tweet.content, 0, 0, model.step)
                for follower in followers
                    push!(follower.feed, retweet)
                end 
                append!(agent.tweets, model.tweet_id)
                push!(model.tweets, tweet)
                model.tweet_id+=1
            end
        else
            retweet=tweet
            if retweet.user_id==agent.id
                return
            else
                tweet=get_tweet_from_id(retweet.retweeted_id, model)
                if abs(agent.opinion-tweet.content) ≤ agent.epsilon_RT
                    tweet.retweet_count+=1
                    MG.add_edge!(model.space.graph, agent.pos, tweet.user_id, :retweet_graph)
                
                    followers_pos=node_neighbors(agent, model, :follower_graph, neighbor_type=:in) 
                    followers=[f for f in allagents(model) if f.pos in followers_pos]
                    for follower in followers
                        push!(follower.feed, Tweet(model.tweet_id, agent.id, true, tweet.id, tweet.user_id, tweet.content, 0, 0, model.step))
                    end 
                    append!(agent.tweets, model.tweet_id)
                    push!(model.tweets, tweet)
                    model.tweet_id+=1
                end
            end
        end
    end
end

# Follow dyamics 
function Follow!(agent,model) 
end

# Unfollow dyamics 
function Unfollow!(agent,model) 
end

# Logout dynamics 
function Logout!(agent,model)
    if rand() ≤ agent.logout_rate
        agent.login=false
    end
end

# Agent dynamics
function agent_step!(agent,model)
    
    Login!(agent,model)
    
    agent.login==false && return 
    
    Read!(agent,model)
    
    Tweet!(agent,model)
    
    Like!(agent,model)
    
    RT!(agent,model)
    
    Logout!(agent,model)

end

agent_step! (generic function with 1 method)

## Parameters

In [8]:
# Population size
const N=100
const nsteps=100

100

## Simulations 

In [9]:
model = @time InitializeModel(N)

agent_data, _ = @time run!(model, agent_step!, model_step!, nsteps, agents_first=true)
                           #replicates=1, parallel=false); 
#or adata = ["opinion"] #@btime for a more accurate timing than @time (uses BenchmarkTools). 
# Warning: it runs the run! function multiple times and then averages: potentially dangerous!

  2.716393 seconds (2.93 M allocations: 153.526 MiB, 2.36% gc time)
119.385768 seconds (1.54 G allocations: 32.269 GiB, 8.85% gc time)


In [10]:
agent=random_agent(model)

User(82, 82, true, Any[1078, 2213, 3171, 3650, 5177, 5579, 6043, 7350, 7498, 7646  …  9306, 9993, 10158, 10430, 11682, 12458, 12767, 13693, 14116, 14450], Any[1], Any[Tweet(1, 77, false, 0, 0, 0.0664663745942673, 15, 215, 0), Tweet(1, 68, false, 0, 0, -0.02053215844743539, 5, 0, 0), Tweet(1, 47, false, 0, 0, 0.417142411837796, 2, 0, 0), Tweet(1, 8, false, 0, 0, 0.764112496200509, 4, 25, 0), Tweet(1, 27, false, 0, 0, -0.39929835204533903, 0, 204, 0), Tweet(1, 95, false, 0, 0, 0.5795744343772591, 0, 61, 0), Tweet(1, 35, false, 0, 0, 0.4682553112739196, 0, 90, 0), Tweet(1, 51, false, 0, 0, -0.2502070931900784, 0, 101, 0), Tweet(1, 60, false, 0, 0, -0.8205616530319029, 2, 75, 0), Tweet(1, 74, false, 0, 0, -0.14058141781844702, 0, 62, 0)  …  Tweet(14496, 15, false, 0, 0, -0.7099665487958053, 0, 0, 99), Tweet(14497, 15, true, 47, 94, -0.6628136795620834, 0, 0, 99), Tweet(14498, 15, true, 67, 63, -0.7035264628696849, 0, 0, 99), Tweet(14499, 15, true, 47, 94, -0.6628136795620834, 0, 0, 99), Tw

In [11]:
allagents(model)

Base.ValueIterator for a Dict{Int64,User} with 100 entries. Values:
  User(68, 68, true, Any[3, 1284, 1285, 1286, 1444, 1445, 1605, 1606, 1739, 174…
  User(2, 2, false, Any[22, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002  …  …
  User(89, 89, false, Any[21, 9741, 9742], Any[1, 64, 65, 66, 97, 137, 138, 200…
  User(11, 11, true, Any[168, 169, 299, 300, 443, 444, 584, 585, 733, 734  …  1…
  User(39, 39, false, Any[76, 301, 586, 859, 1010, 2108, 2372, 3243, 3409, 3713…
  User(46, 46, false, Any[170, 302, 445, 587, 735, 860, 1011, 1135, 1289, 1448 …
  User(85, 85, false, Any[24, 588, 14186], Any[67, 68], Any[Tweet(1, 62, false,…
  User(25, 25, false, Any[2816, 2974, 2975, 9221], Any[32], Any[Tweet(1, 14, fa…
  User(55, 55, true, Any[31, 77, 171, 303, 446, 736, 861, 1012, 1290, 1610  …  …
  User(42, 42, false, Any[32, 172, 304, 589, 862, 1013, 1449, 1611, 1871, 1965 …
  User(29, 29, false, Any[78, 173, 737, 863, 1136, 1291, 1450, 1612, 1966, 2377…
  User(58, 58, true, Any[590, 1014, 3568,

In [12]:
maximum(model.space.graph.favorite_graph.weights)

21

In [13]:
data=sort(DataFrame(allagents(model)), :id)

,id,pos,login,tweets
,Int64,Int64,Bool,Array…
1,1,1,1,"[17, 370, 371, 372, 373, 374, 514, 515, 516, 517, 518, 656, 657, 658, 659, 660, 784, 785, 786, 787, 788, 930, 931, 932, 933, 934, 1057, 1058, 1059, 1060, 1061, 1216, 1217, 1218, 1219, 1369, 1370, 1371, 1372, 1373, 1525, 1526, 1527, 1528, 1529, 1680, 1681, 1682, 1683, 1684, 1809, 1810, 1811, 1812, 1813, 1906, 1907, 1908, 1909, 2048, 2049, 2050, 2051, 2052, 2192, 2193, 2194, 2195, 2196, 2307, 2308, 2309, 2310, 2311, 2445, 2446, 2447, 2448, 2449, 2574, 2575, 2576, 2577, 2578, 2731, 2732, 2733, 2734, 2735, 2895, 2896, 2897, 2898, 2899, 3016, 3017, 3018, 3019, 3020, 3161, 3162, 3163, 3164, 3165, 3320, 3321, 3322, 3323, 3324, 3484, 3485, 3486, 3487, 3488, 3631, 3632, 3633, 3634, 3635, 3777, 3778, 3779, 3780, 3781, 3928, 3929, 3930, 3931, 3932, 4051, 4052, 4053, 4054, 4055, 4219, 4220, 4221, 4222, 4223, 4387, 4388, 4389, 4390, 4391, 4551, 4552, 4553, 4554, 4555, 4706, 4707, 4708, 4709, 4710, 4866, 4867, 4868, 4869, 4870, 5014, 5015, 5016, 5017, 5018, 5158, 5159, 5160, 5161, 5162, 5300, 5301, 5302, 5303, 5304, 5409, 5410, 5411, 5412, 5413, 5561, 5562, 5563, 5564, 5565, 5717, 5718, 5719, 5720, 5721, 5867, 5868, 5869, 5870, 5871, 6031, 6032, 6033, 6034, 6035, 6169, 6170, 6171, 6172, 6173, 6334, 6335, 6336, 6337, 6338, 6484, 6485, 6486, 6487, 6488, 6625, 6626, 6627, 6628, 6629, 6768, 6769, 6770, 6771, 6772, 6890, 6891, 6892, 6893, 6894, 7044, 7045, 7046, 7047, 7048, 7192, 7193, 7194, 7195, 7196, 7338, 7339, 7340, 7341, 7342, 7485, 7486, 7487, 7488, 7630, 7631, 7632, 7633, 7634, 7774, 7775, 7776, 7777, 7778, 7952, 7953, 7954, 7955, 7956, 8101, 8102, 8103, 8104, 8105, 8269, 8270, 8271, 8272, 8273, 8428, 8429, 8430, 8431, 8578, 8579, 8580, 8581, 8582, 8725, 8726, 8727, 8728, 8729, 8873, 8874, 8875, 8876, 8877, 8989, 8990, 8991, 8992, 8993, 9141, 9142, 9143, 9144, 9145, 9295, 9296, 9297, 9298, 9299, 9410, 9411, 9412, 9413, 9414, 9536, 9537, 9538, 9539, 9540, 9662, 9663, 9664, 9665, 9666, 9810, 9811, 9812, 9813, 9814, 9984, 9985, 9986, 9987, 9988, 10145, 10146, 10147, 10148, 10149, 10279, 10280, 10281, 10282, 10283, 10417, 10418, 10419, 10420, 10547, 10548, 10549, 10550, 10551, 10686, 10687, 10688, 10689, 10690, 10812, 10813, 10814, 10815, 10816, 10917, 10918, 10919, 10920, 10921, 11066, 11067, 11068, 11069, 11070, 11215, 11216, 11217, 11218, 11219, 11363, 11364, 11365, 11366, 11367, 11516, 11517, 11518, 11519, 11520, 11668, 11669, 11670, 11671, 11672, 11800, 11801, 11802, 11803, 11804, 11974, 11975, 11976, 11977, 11978, 12125, 12126, 12127, 12128, 12129, 12280, 12281, 12282, 12283, 12284, 12443, 12444, 12445, 12446, 12447, 12597, 12598, 12599, 12600, 12601, 12748, 12749, 12750, 12751, 12752, 12881, 12882, 12883, 12884, 12885, 13051, 13052, 13053, 13054, 13055, 13209, 13210, 13211, 13212, 13213, 13378, 13379, 13380, 13381, 13382, 13536, 13537, 13538, 13539, 13540, 13683, 13684, 13685, 13686, 13687, 13821, 13822, 13823, 13824, 13825, 13955, 13956, 13957, 13958, 13959, 14107, 14108, 14109, 14110, 14111, 14262, 14263, 14264, 14265, 14266, 14439, 14440, 14441, 14442, 14443]"
2,2,2,0,"[22, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 3227, 3228, 3229, 3230, 3231, 3232, 3233, 3234, 3235, 3236, 3237, 3238, 3239, 3240, 3392, 3393, 3394, 3395, 3396, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 5241, 5242, 5243, 5244, 5245, 5246, 5247, 5248, 5249, 5250, 5251, 5252, 5253, 5254, 12962, 12963, 12964, 12965, 12966, 12967, 12968, 12969, 12970, 12971, 12972, 12973, 12974, 12975]"
3,3,3,0,"[1547, 1697, 1824, 1918, 2065, 2323, 2459, 2597, 2911, 3046, 3177, 3342, 3512, 3656, 3805, 3948, 4081, 4250, 4411, 4574, 4882, 5036, 5187, 5315, 5442, 6787, 7649, 7801, 8450, 8601, 9015, 11829, 11995, 12149, 12302, 13240, 13395, 13565, 13699]"
4,4,4,1,"[27, 96, 199, 334, 479, 780, 1052, 1177, 1329, 1485, 1639, 1771, 2010, 2154, 2300, 2406, 2535, 2691, 2857, 3127, 3282, 3447, 3599, 3740, 3890, 4016, 4183, 4514, 5119, 5402, 5521, 5681, 5831, 5992, 6300, 6446, 6589, 7153, 791

## Updates (4-10-2020)

### Activity 
* `neighbors` has been successfully implemented in `MultiplexGraphs.jl`
* `node_neighbors` compatible with `MultiplexGraphs.jl` has been successfully implemented (see *Utilities*)
* Reformed `InitializeModel` with follower graph (still random) initialization 
* Begin thinking about the portfolio of recommendation algorithms 
    * Auxiliary bipartite graph (mono or multi-edge)
    * Weighted sum over multiple edge types

## Updates (5-10-2020)

### Activity 
* Added agent attribute $\epsilon_\ell$ : the opinion distance such that if $|o_i-\rho_i(o_j)| \leq \epsilon_{L} \Rightarrow i$ likes $j$ 's tweet. As a first simple example we'll assume that $\rho_i$ is the identity such that $\rho_i(x)=x$
* Added new attributes (e.g. `attention`, `login_rate`,...)
* Created tweet type
* Implemented powerlaw sampling utility
* Initialized static follower graph, tweet histories and feeds
* Implemented a basic micro dynamics with `Tweet` and `Like!` behaviors
* Simple successful run with `N`=1000 and `nsteps`=100 with a pretty good execution time (the code is not at all optimized!)

## Updates (6-10-2020)

### Activity 
* Added new agent and tweet type attributes (e.g. `favorites`, `favorite_count`, `retweet_count`)
* Implemented `get_tweet_from_id` function 
* Added new attributes to model (e.g. `tweets`: chronologically ordered archive of all tweets)
* Implemented `Read!`, `RT!` behaviors and defined `Login!` and `Logout!` behaviors
* Assumed a an order of magnitude difference between the maximum of *threshold distributions* (e.g. $\epsilon_{L}$, $\epsilon_{RT}$, $\epsilon_{F}$)
* A few corrections to previously implemented behavior functions

## Thoughts & Doubts  
* How to define distinct time scales ($T_{F} \gg T_{RT} > T_{L}$)
* What about defining a status type as a reduced representation of the status object
* Choose `nsteps` taking into account the constraints induced by Monitor limits 
* highlight the latency of the opinion variable: users do not have direct access to the *cognitive opinion* of other but  they have indirect access to *behavioral opinion* leading to a non-zero (tipically high) probability of mis-interpretation (we may model it via a portfolio of probability distributions centered around the true value)
* think about how to implement possibile polarization/segregation mitigation strategies (e.g. *homophily-induced heterophily*)
* Can we learn anything significant from Higgs2012 static and temporal graphs
* How to reliably mine temporal behavioral opinion and reaction mechanisms (e.g. net positive interaction rate: net of T,RTs,L of known polarity / total number of interactions) 

## To-Do List
* Overwrite all graph-related functions in Agents to ensure compatibility with `MultiplexGraphs.jl` (e.g. `get_node_agents()`,...)
* Think about all elementary variables, parameters and timing distributions
* Explore the portfolio of possible recommendation systems (e.g. GNN, bipartite, centrality-based, simple weighted sum,...)
* Implement basic `Follow!` and `RT!` behaviors
* Implement the simplest recommendation system (in `model_step!`)
* Implement non-trivial interpretation distribution $\rho_i$
* Explore and evaluate Higgs2012 static and temporal graph data
* Think of the best parametrization that allows effective calibration 